In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q git+https://github.com/openai/CLIP.git transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 109.2 MB/s eta 0:00:00


In [ ]:
!mkdir /content/data
!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/config.py /content/
!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/utils.py /content/
!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/model.py /content/
!cp -r /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/data /content/

cp: cannot stat '/content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/train.csv': No such file or directory


In [ ]:
!unzip -q /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/videos_train.zip 
!mv videos_train /content/data

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import pickle
import clip
import gdown
import zipfile

from utils import *
from model import *
from config import CFG

In [ ]:
def preprocessing(cur_df, save_path):
    all_frames = []
    all_captions = []
    i = 0
    for caption, video_name in tqdm(zip(cur_df.caption, cur_df.paths), total=len(cur_df)):
        name = f'{CFG.video_path}/{video_name}'
        text = f'Caption: {caption}<|endoftext|>'

        if os.path.exists(name):
            
            video = read_video(path = name, frames_num=1)

            all_frames.append(video[0])
            all_captions.append(text)

    with open(save_path, 'wb') as f:
        pickle.dump({"clip_embedding": all_frames, "captions": all_captions}, f)

df = pd.read_csv(CFG.main_df_path, index_col=0)


df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# valid
valid_df.to_csv(CFG.valid_df_path, index=False)
preprocessing(valid_df, CFG.valid_features_path)

# train
df.to_csv(CFG.train_df_path, index=False)
preprocessing(df, CFG.train_features_path)

100%|██████████| 4520/4520 [35:05<00:00,  2.15it/s]


In [ ]:
!cp -r /content/data /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/